In [2]:
import jax
import jax.numpy as jnp
from jax import random
import math

In [3]:
try:
    import flax
except ModuleNotFoundError: # Install flax if missing
    !pip install --quiet flax
    import flax

from flax import linen as nn

In [4]:
class InputEmbeddings(nn.Module):
    model_dimension : int
    vocab_size : int

    def setup(self):
        self.embedding = nn.Embed(num_embeddings=self.vocab_size, features=self.model_dimension)
    
    def __call__(self, x):
        x = self.embedding(x) * math.sqrt(self.model_dimension)
        return x

In [5]:
class FeedForward(nn.Module):
    model_dimension : int
    ff_dim : int
    dropout : float

    def setup(self):
        self.linear1 = nn.Dense(features=self.ff_dim)
        self.linear2 = nn.Dense(features=self.model_dimension) 
    
    def __call__(self, x):
        x = self.linear1(x)
        x = nn.relu(x)
        x = self.linear2(x)
        x = nn.relu(x)
        return x

In [ ]:
#defining single head of attention
def ScaledDotProduct(q, k, v):
    dk = q.shape[-1]
    kT = jnp.transpose(k, (0, 2, 1))
    weights = jnp.matmul(q, kT)
    size = weights.shape[-1]
    weights = weights/math.sqrt(dk)
    mask = jnp.tril(jnp.ones((size, size)))
    logits = jnp.where(mask == 0, -9e15, weights)
    attention = nn.softmax(logits, axis=-1)
    values = jnp.matmul(attention, v)
    return values 
    

In [8]:
#einsum version
#defining single head of attention
def ScaledDotProduct(q, k, v):
    dk = q.shape[-1]
    weights = jnp.einsum('b t c, b T c -> b t T', q, k) / math.sqrt(dk)
    size = weights.shape[-1]
    mask = jnp.tril(jnp.ones((size, size)))
    logits = jnp.where(mask == 0, -9e15, weights)
    attention = nn.softmax(logits, axis=-1)
    values = jnp.einsum('b t T, b T c -> b t c', attention, v)
    return values 

In [ ]:
class MultiHeadAttention(nn.Module):
    

In [ ]:
class PositionalEmbeddings(nn.Module):
    model_dimension : int
    seq_len : int
#doto

In [ ]:
#MH Attention
#Feedforward